# Pumped-Storage Optimisation with Neural Network trained by Genetic Algorithm

In [1]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_ANN, evaluate_fitness

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

# Import Pytorch stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

# Import genetic algorithm library
from deap import base, creator, tools, algorithms

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## The Power Plant

In [2]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]
plant_params["LOOKAHEAD"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * 3

## Price Data

In [3]:
df = pd.read_csv("../01 - Data/spot_prices_utc.csv").assign(utc_time = lambda x: pd.to_datetime(x.utc_time))
df.head(2)

,spot,utc_time
0,36.99,2017-12-31 23:00:00+00:00
1,31.08,2018-01-01 00:00:00+00:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48166 entries, 0 to 48165
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   spot      48166 non-null  float64            
 1   utc_time  48166 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 752.7 KB


In [5]:
df.utc_time.min()

Timestamp('2017-12-31 23:00:00+0000', tz='UTC')

In [6]:
df.utc_time.max()

Timestamp('2023-06-30 21:00:00+0000', tz='UTC')

Keep the part that was before the madness:

In [7]:
df = df.query('utc_time < @pd.Timestamp("2020-01-01", tz="UTC")')

Then set two weeks aside, one for validation and one for testing.

In [8]:
df_train = df.query('utc_time < @pd.Timestamp("2019-12-18", tz="UTC")').reset_index(
    drop=True
)
df_val = (
    df.query('utc_time >= @pd.Timestamp("2019-12-18", tz="UTC")')
    .query('utc_time < @pd.Timestamp("2019-12-25", tz="UTC")')
    .reset_index(drop=True)
    .reset_index(drop=True)
)
df_test = df.query('utc_time >= @pd.Timestamp("2019-12-25", tz="UTC")').reset_index(
    drop=True
)

In [9]:
print(df_train.shape, df_val.shape, df_test.shape)

(17184, 2) (168, 2) (168, 2)


### Preparing the dataset for the neural network

In [10]:
df_train

,spot,utc_time
0,36.99,2017-12-31 23:00:00+00:00
1,31.08,2018-01-01 00:00:00+00:00
2,29.17,2018-01-01 01:00:00+00:00
3,21.96,2018-01-01 02:00:00+00:00
4,14.96,2018-01-01 03:00:00+00:00
...,...,...
17179,47.55,2019-12-17 19:00:00+00:00
17180,41.18,2019-12-17 20:00:00+00:00
17181,41.53,2019-12-17 21:00:00+00:00
17182,39.84,2019-12-17 22:00:00+00:00


Create maximum number of possible 168 hour sequences and put them into table format:

In [11]:
def create_sequences(in_array, sequence_length):
    out_array = np.zeros((len(in_array)-sequence_length+1, sequence_length), dtype=np.float32)

    for i in tqdm(range(len(in_array)-sequence_length+1)):
        out_array[i] = np.array(in_array[i:i+sequence_length])

    return out_array

In [12]:
len(df_train.spot)

17184

In [13]:
X_train = create_sequences(df_train.spot, 168)
X_val = create_sequences(df_val.spot, 168)
X_test = create_sequences(df_test.spot, 168)

  0%|          | 0/17017 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
X_train

array([[36.99, 31.08, 29.17, ..., 42.58, 45.05, 33.3 ],
       [31.08, 29.17, 21.96, ..., 45.05, 33.3 , 21.95],
       [29.17, 21.96, 14.96, ..., 33.3 , 21.95, 19.67],
       ...,
       [42.43, 37.36, 34.94, ..., 47.55, 41.18, 41.53],
       [37.36, 34.94, 33.57, ..., 41.18, 41.53, 39.84],
       [34.94, 33.57, 32.8 , ..., 41.53, 39.84, 38.81]], dtype=float32)

In [15]:
X_train.shape

(17017, 168)

In [16]:
X_val

array([[35.45, 34.29, 33.08, 33.01, 36.92, 43.47, 48.56, 50.73, 50.4 ,
        49.35, 48.61, 48.03, 47.72, 48.62, 50.08, 52.22, 54.13, 54.17,
        50.99, 47.76, 41.58, 43.58, 39.33, 36.19, 32.3 , 30.13, 29.64,
        28.44, 34.67, 43.03, 44.05, 46.52, 46.3 , 46.16, 45.1 , 44.1 ,
        44.07, 45.69, 46.57, 47.39, 46.08, 45.67, 45.59, 43.02, 41.05,
        40.02, 37.02, 33.72, 32.08, 30.16, 29.88, 29.96, 33.6 , 39.03,
        44.71, 42.08, 38.81, 44.38, 44.77, 42.68, 42.47, 42.98, 43.07,
        37.6 , 38.58, 37.96, 40.33, 38.59, 38.32, 35.1 , 31.23, 31.97,
        26.83, 21.92, 20.56, 20.48, 21.66, 30.07, 33.17, 33.93, 35.1 ,
        35.28, 35.72, 33.3 , 33.79, 33.94, 35.96, 37.76, 39.02, 40.01,
        37.56, 35.27, 34.2 , 32.47, 24.5 , 21.18, 13.93,  8.88,  7.39,
         6.49,  7.91,  9.13, 16.52, 22.22, 28.48, 29.99, 31.39, 32.15,
        29.87, 30.89, 31.74, 35.3 , 38.34, 37.49, 37.1 , 33.77, 29.58,
        27.34, 21.93, 14.17,  9.81,  6.08,  3.1 ,  3.78,  8.84, 24.29,
      

In [17]:
X_val.shape

(1, 168)

The different columns represent the hour 1 to hour 168, but indexed at zero, hence only until 167. Each row will be fed into the neural network.

In [18]:
pd.DataFrame(X_train).head(3)

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
0,36.990002,31.080000,29.170000,21.959999,14.96,-5.92,-6.10,-10.02,-5.71,-1.88,...,20.660000,20.299999,24.150000,32.270000,46.389999,46.919998,45.450001,42.580002,45.049999,33.299999
1,31.080000,29.170000,21.959999,14.960000,-5.92,-6.10,-10.02,-5.71,-1.88,-10.48,...,20.299999,24.150000,32.270000,46.389999,46.919998,45.450001,42.580002,45.049999,33.299999,21.950001
2,29.170000,21.959999,14.960000,-5.920000,-6.10,-10.02,-5.71,-1.88,-10.48,-6.79,...,24.150000,32.270000,46.389999,46.919998,45.450001,42.580002,45.049999,33.299999,21.950001,19.670000


## MILP

Find the solutions for the validation and testing periods.

In [19]:
milp_solver = MILP(plant_params=plant_params, spot=X_train[0], utc_time=df_val["utc_time"])
milp_model, milp_status, milp_profile = milp_solver.solve()
print(milp_status)
optimal_val = milp_model.objective.value()
print(optimal_val)
milp_profile.head(2)

Optimal
95261.00076436996


,water_level,action,colour_id,utc_time,spot
0,0.0,0,nothing,2019-12-18 00:00:00+00:00,36.990002
1,0.0,0,nothing,2019-12-18 01:00:00+00:00,31.080000


In [20]:
milp_solver = MILP(plant_params=plant_params, spot=df_val["spot"], utc_time=df_val["utc_time"])
milp_model, milp_status, milp_profile = milp_solver.solve()
print(milp_status)
optimal_val = milp_model.objective.value()
print(optimal_val)
milp_profile.head(2)

Optimal
46943.0


,water_level,action,colour_id,utc_time,spot
0,375.0,-1,pump,2019-12-18 00:00:00+00:00,35.45
1,750.0,-1,pump,2019-12-18 01:00:00+00:00,34.29


In [21]:
milp_solver = MILP(plant_params=plant_params, spot=df_test["spot"], utc_time=df["utc_time"])
milp_model, milp_status, milp_profile = milp_solver.solve()
print(milp_status)
optimal_test = milp_model.objective.value()
print(optimal_test)
milp_profile.head(2)

Optimal
43665.0


,water_level,action,colour_id,utc_time,spot
0,375.0,-1,pump,2017-12-31 23:00:00+00:00,7.50
1,750.0,-1,pump,2018-01-01 00:00:00+00:00,5.47


## Feed Forward Neural Network

Predict the full week (168 output neurons, sigmoid activation to output values between 0 and 1, then use thresholds for translations into actions).

Randomly initiate the lake level such that the network can learn different situations.

In [22]:
class ANN(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, hidden_size):
        super(ANN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layers = hidden_layers
        self.hidden_size = hidden_size
        
        # Define input layer
        self.input_layer = nn.Linear(input_size, hidden_size)
        
        # Define hidden layers
        self.hidden_layers = nn.ModuleList()
        for _ in range(hidden_layers):
            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
        
        # Define output layer
        self.output_layer = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

In [23]:
model = ANN(input_size=168, output_size=168, hidden_layers=1, hidden_size=16)
for param in model.parameters():
    param.requires_grad = False

model

ANN(
  (input_layer): Linear(in_features=168, out_features=16, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=16, out_features=16, bias=True)
  )
  (output_layer): Linear(in_features=16, out_features=168, bias=True)
)

In [24]:
def encode_chromosome(model):
    """
    Encodes the model parameters into a single chromosome, which can be used for genetic algorithms.
    """
    chromosome = torch.tensor([], dtype=torch.float32)

    for param in model.parameters():
        chromosome = torch.cat((chromosome, param.view(-1)))

    return chromosome.numpy()


def decode_chromosome(model, chromosome):
    """
    Decodes the chromosome into the model parameters and updates the model with the new parameters.
    """
    chromosome = torch.tensor(chromosome)
    model_params = list(model.parameters())
    start = 0

    for param in model_params:
        end = start + torch.numel(param)
        param.data = chromosome[start:end].view(param.size())
        start = end

In [25]:
individual = encode_chromosome(model)
individual

array([ 0.02309757, -0.06416433, -0.02658171, ..., -0.21808612,
       -0.1612595 ,  0.06143069], dtype=float32)

Then setting the weights manually using a chromosome (example all zeros):

In [26]:
random_new_chromosome = np.zeros_like(individual, dtype=np.float32)

decode_chromosome(model, random_new_chromosome)
list(model.parameters())

[Parameter containing:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 Parameter containing:
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [27]:
next(model.parameters()).dtype

torch.float32

In [28]:
X_train[0].dtype

dtype('float32')

In [29]:
model(torch.tensor(X_train[0]))

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 

In [30]:
evaluate_fitness(
    population=[model(torch.tensor(X_train[0])).detach().numpy()],
    ps_params=plant_params,
    spot_prices=X_train[0],
)

array([0])

### Developing the genetic algorithm


In [31]:
model = ANN(input_size=169, output_size=168, hidden_layers=1, hidden_size=16)

# Need to set the gradients to zero
for param in model.parameters():
    param.requires_grad = False

Initialise a population of random individuals (200) of the size of parameters ():

In [32]:
population_size = 200
individual_size = encode_chromosome(model).shape[0]

population = np.random.normal(loc=0, scale=0.1, size=(population_size, individual_size))
population = np.array(population, dtype=np.float32)
population

array([[ 0.11089212,  0.0997879 , -0.1399377 , ...,  0.017062  ,
         0.11972897,  0.05528555],
       [ 0.13017757,  0.04601926,  0.06193453, ...,  0.02769292,
         0.14260024, -0.1349315 ],
       [-0.07609696,  0.11903714,  0.19870432, ..., -0.24225171,
         0.16072379, -0.07765348],
       ...,
       [-0.04571274, -0.02426161,  0.06360061, ...,  0.17606959,
        -0.05138629,  0.04703814],
       [ 0.01624773,  0.0976718 ,  0.04584014, ..., -0.08011267,
        -0.17762926, -0.08881866],
       [-0.06859829, -0.17922966,  0.04481868, ..., -0.15205441,
         0.05462956, -0.17786144]], dtype=float32)

Make predictions with each individual, which will then be used to evaluate their fitness:

In [33]:
prices = df.spot
lake_level = 2500
# add the lake level to input data
input_data = torch.cat([torch.tensor([lake_level]), torch.tensor(X_val[0])])

predictions = np.zeros((population_size, X_val.shape[1]), dtype=np.float32)

for idx, individual in enumerate(population):
    
    # Set the model weights
    decode_chromosome(model, individual)

    # Make predictions with the configuration
    predictions[idx] = model(input_data).numpy()

Evaluate the fitness of each individual using their predictions:

In [34]:
fitnesses = evaluate_fitness(
    population=predictions,
    ps_params=plant_params,
    spot_prices=X_val[0]
)

fitnesses[:10]

array([-2667526.99882984, -2603722.99954891, -2353027.99885273,
       -3184122.99835682, -2069011.99953556, -3668374.99992847,
       -4483544.00031567, -2381706.99837208, -3957159.99891758,
       -4643255.99925518])

Gaussian mutation:

In [35]:
population[0] + np.random.normal(loc=0, scale=0.1, size=individual_size)

array([ 0.01231367,  0.18103372, -0.30211253, ..., -0.01691937,
       -0.01462722,  0.05070179])

## Using the Class

In [38]:
best_config = {
    "TOTAL_GENERATIONS": 500,
    "POP_SIZE": 200,
    "INITIAL_MUTATION_RATE": 0.28,
    "FINAL_MUTATION_RATE": 0.09,
    "MUTATION_SIGMA": 4,
    "INITIAL_EXPLORATION": 0.67,
    "ELITISM": 0.5,
    "SURVIVAL_RATE": 0.5,
}

In [39]:
ga_solver = GA_ANN(
    plant_params=plant_params,
    spot=X_val[0],
    hidden_layers=3,
    hidden_size=8
)

population, fitnesses, history = ga_solver.train(
    config=best_config,
    tune_mode=False
)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
history

In [ ]:
(
    pn.ggplot(
        data=history,
        mapping=pn.aes(x="generation", y="best"),
    )
    + pn.geom_line()
    + pn.theme(figure_size=[6, 3])
    # + pn.scale_y_log10()
)